In [1]:
# Mount Google Drive to access your PDFs
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!sudo apt-get install poppler-utils -y
!pip3 install colpali-engine vidore_benchmark pdf2image google-generativeai pypdf pyvespa vespacli requests PyPDF2

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 30 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.7 [186 kB]
Fetched 186 kB in 2s (119 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package poppler-utils.
(Reading database ... 126315 

In [3]:
import torch
import numpy as np
import os
import base64
import requests
from PIL import Image
from io import BytesIO
from typing import Dict, List

import torch

from torch.utils.data import DataLoader
from torchvision import transforms

from colpali_engine.models import ColPaliProcessor
from colpali_engine.models import ColPali, ColPaliProcessor
from colpali_engine.utils.torch_utils import get_torch_device

from vespa.application import Vespa
from vespa.io import VespaQueryResponse
from vespa.deployment import VespaCloud

from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import AutoModel, AutoTokenizer
from transformers import AutoProcessor, Gemma3ForConditionalGeneration
from transformers import pipeline


In [4]:
# Read the API key from your file
with open('/content/drive/MyDrive/Colab_Secrets/huggingface_token.txt', 'r') as file:
    hf_token = file.read().strip()

# Set it as an environment variable
os.environ['HUGGINGFACE_TOKEN'] = hf_token

hf_token = os.environ["HUGGINGFACE_TOKEN"]
login(token=hf_token)

In [5]:
# Read the API key from your file
with open('/content/drive/MyDrive/Colab_Secrets/vespa_token.txt', 'r') as file:
    api_key = file.read().strip()

# Set it as an environment variable
os.environ['VESPA_CLOUD_SECRET_TOKEN'] = api_key


vespa_cloud_kB97qepLwwKI6BwCr8Yw3nmdfn74O3XlLCzfieDUWol


In [9]:
token_endpoint = "https://e7c888f9.e41c6780.z.vespa-app.cloud/"

In [11]:
app = Vespa(
    url=token_endpoint, vespa_cloud_secret_token=os.getenv("VESPA_CLOUD_SECRET_TOKEN")
)

In [12]:
app.get_application_status()

<Response [200]>

In [13]:
device = get_torch_device("auto")
if device == "cuda":
    dtype = torch.bfloat16
else:
    dtype = torch.float32

In [14]:
model_name = "vidore/colpali-v1.2"
model = ColPali.from_pretrained(model_name, torch_dtype=dtype, device_map=device).eval()
processor = ColPaliProcessor.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/750 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/66.3k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

...

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/700 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/243k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.8M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/733 [00:00<?, ?B/s]

In [15]:
def decode_base64_image(base64_string):
    """
    Decodes a Base64 string into a PIL Image object.
    """
    image_data = base64.b64decode(base64_string)
    return Image.open(BytesIO(image_data))

In [16]:
def float_query_token_vectors(vectors: torch.Tensor) -> Dict[str, List[float]]:
    """
    Convert query token vectors to Vespa-compatible format.

    Args:
        vectors (torch.Tensor): Query token vectors

    Returns:
        Dict[str, List[float]]: Dictionary with token indices as keys and float vectors as values
    """
    vespa_token_dict = dict()
    for index in range(0, len(vectors)):
        vespa_token_dict[str(index)] = vectors[index].tolist()
    return vespa_token_dict




In [17]:
def process_queries(app, queries, qs, top_k=2, top_pages=2):
    """
    Processes multiple queries, queries the Vespa app, and returns a dictionary with query, images, and scores.

    Args:
        app: The Vespa app instance.
        queries: A list of query strings.
        qs: A list of query token vectors corresponding to the queries.
        top_k: The number of top hits to process for each query.
        top_pages: The number of best matching pages to process for each hit.

    Returns:
        A list of dictionaries, each containing 'query', 'images', and 'scores'.
    """

    final_result = {
        'queries' :[],
        'images' :[],
        'scores':[]
    }
    for idx, query in enumerate(queries):

        try:
        # Query the Vespa app
            response = app.query(
                yql="select images from doc where userInput(@userQuery)",
                ranking="default",
                userQuery=query,
                timeout=2,
                hits=top_k,
                body={
                    "presentation.format.tensors": "short-value",
                    "input.query(qt)": float_query_token_vectors(qs[idx]),
                },
            )
            assert response.is_successful()

        except:
            print(f"Error querying: {str(e)}")

        # Initialize the result for this query
        query_result = {"query": query, "images": [], "scores": []}

        for hit in response.hits[:top_k]:  # Process top_k hits
            try :
                match_scores = hit["fields"]["matchfeatures"]["max_sim_per_page"]
                images = hit["fields"]["images"]

                # Find the top matching pages
                sorted_pages = sorted(match_scores.items(), key=lambda x: x[1], reverse=True)
                best_pages = sorted_pages[:top_pages]

                for page, score in best_pages:
                    try:
                        image_data = base64.b64decode(images[int(page)])

                        # Append the image and score to the query result
                        query_result["images"].append(image_data)
                        query_result["scores"].append(score)

                    except Exception as e:
                        print(f"Error processing best page in hit: {str(e)}")
                        # Append empty lists in case of an error
                        query_result["images"].append([])
                        query_result["scores"].append([])
            except:
                print(f"Error processing hit: {str(e)}")
                for _ in range(len(best_pages)):
                    query_result["images"].append([])
                for _ in range(len(best_pages)):
                    query_result["scores"].append([])

        if len(query_result["images"]) != len(query_result["scores"]):
            raise ValueError("Mismatch: 'images' and 'scores' do not have the same length in query result for a hit.")
        else:
            final_result["queries"].append(query_result["query"])
            final_result["images"].append(query_result["images"])
            final_result["scores"].append(query_result["scores"])
    # Sanity check for matching lengths of queries, images, and scores
    if not (len(final_result["queries"]) == len(final_result["images"]) == len(final_result["scores"])):
        raise ValueError("Mismatch: 'queries', 'images', and 'scores' do not have the same length in final_result")
    else:
        return final_result



In [18]:
queries = ["Easiest way to pay taxes", "when is the tax deadline", "where to send my tax returns"]

In [19]:
dataloader = DataLoader(
    queries,
    batch_size=1,
    shuffle=False,
    collate_fn=lambda x: processor.process_queries(x),
)
qs = []
for batch_query in dataloader:
    with torch.no_grad():
        batch_query = {k: v.to(model.device) for k, v in batch_query.items()}
        embeddings_query = model(**batch_query)
        # also handle if running on GPU
        if model.device == "cuda":
            embeddings_query = embeddings_query.float()
        qs.extend(list(torch.unbind(embeddings_query.cpu())))

In [20]:
results = process_queries(app, queries, qs)

In [21]:
import pickle
output_dir = "/content/drive/MyDrive/Projects/RagTaxHelper"
output_file = os.path.join(output_dir, "test_out.pkl")

# Save the updated dictionary to the specified path
with open(output_file, "wb") as f:
    pickle.dump(results, f)

print(f"Dictionary with entropies saved successfully to {output_file}")

Dictionary with entropies saved successfully to /content/drive/MyDrive/Projects/RagTaxHelper/test_out.pkl


In [ ]:
pipe = pipeline(
    "image-text-to-text",
    model="google/gemma-3-4b-it", # "google/gemma-3-12b-it", "google/gemma-3-27b-it"
    device="cuda",
    torch_dtype=torch.bfloat16
)



In [ ]:
def convert_images_to_base64(image_data_list):
    """
    Converts a list of raw image data (binary) to base64-encoded strings.

    Args:
        image_data_list (list): A list of raw image data (binary).

    Returns:
        list: A list of base64-encoded image strings.
    """
    base64_images = []
    for image_data in image_data_list:
        # Ensure the input is in bytes
        if not isinstance(image_data, (bytes, bytearray)):
            raise TypeError(f"Expected bytes-like object, got {type(image_data).__name__}")

        # Process the image
        buffered = BytesIO()
        image = Image.open(BytesIO(image_data))  # Open the image from raw binary data
        image.save(buffered, format="PNG")  # Save the image to a buffer in PNG format
        img_str = base64.b64encode(buffered.getvalue()).decode()  # Convert to base64 string
        base64_images.append(img_str)
    return base64_images

In [ ]:
def build_messages_with_context(query_result):
    """
    Builds a messages structure for a chatbot based on queries and images,
    with additional context added to each query.

    Args:
        query_result (dict): A dictionary containing 'queries' and 'images'.
            - 'queries': List of user queries (strings).
            - 'images': Nested list of raw image data (binary data).
        convert_images_to_base64 (function): A function to convert raw image data to base64 strings.

    Returns:
        None
    """
    # Initialize the messages list with the system message
    messages = [
        {
            "role": "system",
            "content": [{"type": "text", "text": "You are a helpful assistant, that uses both the images provided and your own knowledge to answer any questions"}]
        }
    ]

    # Iterate over queries and images to build the user messages
    for query, images in zip(query_result["queries"], query_result["images"]):
        # Add context to the query
        modified_query = (
            f"This is a question about the tax system in the USA: {query} "
            f"Here are some relevant images from IRS forms. Please give a comprehensive answer."
        )

        # Convert the nested list of raw image data to base64 strings
        base64_images = convert_images_to_base64(images)

        # Build the content for this query
        content = [{"type": "text", "text": modified_query}] + [
            {"type": "image", "url": f"data:image/png;base64,{img_str}"}
            for img_str in base64_images
        ]

        # Append the user message
        messages.append({
            "role": "user",
            "content": content
        })

        # Generate the assistant's response for this query
        output = pipe(text=messages, max_new_tokens=200)

        # Extract the assistant's response
        assistant_response = output[0]["generated_text"][-1]["content"]

        # Append the assistant's response to maintain alternation
        messages.append({
            "role": "assistant",
            "content": [{"type": "text", "text": assistant_response}]
        })

        print("-"*50)

        print(f"Query: {query}")
        print("")
        print(f"Assistant Response:")
        print("")
        # Print the assistant's response
        print(assistant_response)

In [ ]:
build_messages_with_context(results)